# Simple LSTM

## 1) data 는 0 ~ 99 까지의 연속된 숫자이고, target 은 (1 ~ 101) * 2 으로 구성한다. 입력  data 에 대응하는 출력 data 를 예측하는 model 을 LSTM 으로 작성

## 연속된 5 개의 숫자를 보고 다음 숫자를 알아맞추도록 LSTM 을 이용한 model 작성

시퀀스 데이터는 순서를 섞으면 안됨!

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

### Training Data 작성 요령

In [3]:
numbers = [[i] for i in range(105)] # 앞에 5개를 우리가 input 데이터로 100개를 만들려고 하면
# 앞에 5개가 첫 번째 input이 되기 때문에 감안해서 105개까지 데이터를 만들었음
numbers[:10] # 리스트 안에 리스트가 있는 형태

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

시퀀스 데이터 생성하기

In [5]:
data = []
target = []
# 4개의 연속된 숫자로 그 다음 5번째를 알아 맞추는거임 --> 5개 필요
for i in range(5, len(numbers)): # i가 5부터 시작했으니깐 0부터 시작하려면 i-5를 해준다.
  data.append(numbers[i-5 : i]) # 0,1,2,3,4가 data에 추가
  target.append(numbers[i][0]*2) # 리스트 안에 리스트가 있어서 [i][0]

data[:5] # 시퀀스 데이터를 만들었음

[[[0], [1], [2], [3], [4]],
 [[1], [2], [3], [4], [5]],
 [[2], [3], [4], [5], [6]],
 [[3], [4], [5], [6], [7]],
 [[4], [5], [6], [7], [8]]]

In [6]:
target[:5] # 첫 번째 행 0,1,2,3,4 다음에는 5가 들어와야하는데 그 2배인 10이 target

[10, 12, 14, 16, 18]

현재 데이터는 그냥 순수한 파이썬 리스트 --> 이를 그냥 쓸 수 없기에 numpy array로 바꿔야함.

In [16]:
data = np.array(data, dtype = "float32")
target = np.array(target, dtype = "float32")

data = data / 100.
target = target / 100. # 소수점으로 스케일
data.shape, target.shape
# batch size = 100, 타임 스텝 : 5, 세 번째 인자, 피처 = 1
# 지금은 피처를 하나만 만들었음.

((100, 5, 1), (100,))

# 모델 생성

In [17]:
model = Sequential()
model.add(LSTM(16,input_shape = (5,1)))
model.add(Dense(1)) # 연속된 숫자를 맞추는 회귀 문제이므로 activation X
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 16)                  │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,169 (4.57 KB)

 Trainable params: 1,169 (4.57 KB)

 Non-trainable params: 0 (0.00 B)

# 모델 컴파일

In [18]:
model.compile(optimizer= 'adam', loss = 'mae', metrics = ['mae'])

# 모델 학습

In [19]:
 # 데이터가 작아서 훈련, 테스트로 데이터 분할을 안 해줬음
 # 만약 중간중간 validation을 하고 싶으면
 model.fit(data,target,epochs = 500,validation_split = 0.2)
 # validaion_split : data,target을 매 에포크마다 임의로 8 : 2로 나눈다
 # 80%는 훈련용, 20%는 검증용

Epoch 1/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - loss: 0.8160 - mae: 0.8160 - val_loss: 1.7182 - val_mae: 1.7182
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.8021 - mae: 0.8021 - val_loss: 1.6793 - val_mae: 1.6793
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.7835 - mae: 0.7835 - val_loss: 1.6398 - val_mae: 1.6398
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.7726 - mae: 0.7726 - val_loss: 1.5995 - val_mae: 1.5995
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.7561 - mae: 0.7561 - val_loss: 1.5581 - val_mae: 1.5581
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.6892 - mae: 0.6892 - val_loss: 1.5156 - val_mae: 1.5156
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.6723 - mae: 0.6723 - val_loss: 1.4714 - val_mae: 1.4714
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.6006 - mae: 0.6006 - val_loss: 1.4254 - val_mae: 1.4254
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.6106 - mae:

In [20]:
test_data=[[35],[36],[37],[38],[39]] # 연속적인 숫자로 패턴을 인식했음
# batch_size를 생각해서 (1,5,1)로 reshape을 해줘야한다.
x = np.array(test_data, dtype = "float32").reshape(1,5,1) / 100.# shape을 맞춰야함
# 테스트 데이터도 스케일링 해줘야한다.
# 현재는 소수점으로 스케일링 되어 있어서 다시 100을 곱해서 되돌린다.
model.predict(x) * 100 # 80이 나올 것이라고 기대했는데 32가 나옴
# 데이터 사이즈가 0~100까지 편차가 크다 --> 스케일링 해주기 --> 다시 100 곱해주

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step


array([[80.02383]], dtype=float32)